# Example: Potjans & Diesmann microcircuit  2014

This example notebook submits a job to a chosen HPC backend of EBRAINS/HBP (**JUSUF**), loads a **Singularity** container with a complete **hybridLFPy** installation and executes the hybridLFPy example file ``example_microcircuit.py`` (https://github.com/INM-6/hybridLFPy/blob/master/examples/example_microcircuit.py) in parallel on the backend.

For more in-depth info on using HPC backends via EBRAINS see https://wiki.ebrains.eu/bin/view/Collabs/using-supercomputers-from-the-collab/

## Prepare connection to backend

In [1]:
# use the pyunicore library
!pip install pyunicore --upgrade

  Created wheel for pyunicore: filename=pyunicore-0.9.1-py3-none-any.whl size=13319 sha256=1007d49e852428ca7333956e7db76c50728daeea0dccd681b97ed4ac3c063e75
  Stored in directory: /tmp/pip-ephem-wheel-cache-yngknpq5/wheels/df/c3/a4/8ea35f2317f259418ef5f5564921fdffb895a3a4fc2ff7b2b6
Successfully built pyunicore
  Attempting uninstall: pyunicore
    Found existing installation: pyunicore 0.5.7
    Uninstalling pyunicore-0.5.7:
      Successfully uninstalled pyunicore-0.5.7
You should consider upgrading via the '/opt/app-root/bin/python3 -m pip install --upgrade pip' command.


In [2]:
# import modules
import os
import pyunicore.client as unicore_client
import requests
import json
from time import sleep, time
from pprint import pprint
from IPython.display import IFrame, clear_output

In [3]:
# Create connection to supercomputer (i.e., JUSUF)
tr = unicore_client.Transport(clb_oauth.get_token())
r = unicore_client.Registry(tr, unicore_client._HBP_REGISTRY_URL)

In [4]:
# Valid choices for supercomputers are one of the keys in:
r.site_urls

{'RWTH': 'https://unicore.hpc.itc.rwth-aachen.de:8080/RWTH/rest/core',
 'FZJ_JURECA': 'https://zam2125.zam.kfa-juelich.de:9112/FZJ_JURECA/rest/core',
 'UNICORE-TEST': 'https://catto.cscs.ch:8080/UNICORE-TEST/rest/core',
 'DAINT-CSCS': 'https://brissago.cscs.ch:8080/DAINT-CSCS/rest/core',
 'irene': 'https://hbp-unicore.ccc.cea.fr/irene/rest/core',
 'JUWELS': 'https://zam2125.zam.kfa-juelich.de:9112/JUWELS/rest/core',
 'JUSUF': 'https://zam2125.zam.kfa-juelich.de:9112/JUSUF/rest/core',
 'JUDAC': 'https://zam2125.zam.kfa-juelich.de:9112/JUDAC/rest/core',
 'CINECA-GALILEO': 'https://grid.hpc.cineca.it:9111/CINECA-GALILEO/rest/core',
 'BSC-MareNostrum': 'https://unicore-hbp.bsc.es:8080/BSC-MareNostrum/rest/core',
 'CINECA-MARCONI': 'https://grid.hpc.cineca.it:9111/CINECA-MARCONI/rest/core'}

In [5]:
supercomputer = 'JUSUF'
try:
    site_client = r.site(supercomputer)
except KeyError:
    # cluster may be dropped from Registry.site_urls for whatever reason
    site_client = unicore_client.Client(r.transport, 'https://zam2125.zam.kfa-juelich.de:9112/JUSUF/rest/core')

In [6]:
# check connection to supercomputer
headers = {}
headers["Authorization"] = "Bearer " + clb_oauth.get_token()
headers['Accept'] = "application/json"
rs = requests.get(url=site_client.site_url, headers = headers, verify = False)
print("Status code %s " % rs.status_code)
print("Content-type %s " % rs.headers['Content-Type'])
reply = rs.json()
# print(json.dumps(reply, indent = 1))

Status code 200 
Content-type application/json 


## Prepare singularity container
See https://apps.fz-juelich.de/jsc/hps/jusuf/cluster/container-runtime.html#container-runtime-on-system-name for details. 

This step builds the singularity container. It is optional if the recipe has already been uploaded and built on the system. 

The procedure may be different on different HPC backends. 
The container can either way be built from the same recipe: https://raw.githubusercontent.com/INM-6/hybridLFPy/master/Dockerfile

Turn the below Raw block into Code in order to run:

## Prepare main simulation job
This step combines in a single session the following:

- download hybridLFPy container
- download simulation files from GitHub (https://github.com/INM-6/hybridLFPy)
- ask for resources (# nodes, # MPI processes, # seconds runtime)
- execute simulation via the batch job queue
- download tar file with simulation output

### Notes

Singularity exposes the host environment to the running container (i.e., ``$PATH``, ``$PYTHONPATH``, ``$HOME`` etc.). 
The container includes Python 3.8.5, and may find pip-installed software in your host's ``$HOME/.local/lib/python3.8/site-packages`` directory and similar if it exist. 

If you encounter problems, try including ``unset PYTHONPATH`` in your ``simulation_job["Executable"]`` part below.

In [7]:
# create dictionary with job specification and define list of input files from this Collab
# simulation_job = {"Job type": "interactive"}
simulation_job = {}
simulation_inputs = []

In [8]:
# Resources
simulation_job['Resources'] = {
    "Queue": "batch",
    "Nodes": "4",
    "CPUsPerNode": "128",
    #"CPUs": "512",
    "Runtime": "3600",
}

In [9]:
# commands run on login node before job execution
simulation_job["User precommand"] = """module use $OTHERSTAGES
module --force purge
module load Stages/2020
module load GCC Singularity-Tools git

git clone --branch master https://github.com/INM-6/hybridLFPy.git
cd hybridLFPy/examples
sib download --recipe-name hybridlfpy
"""
simulation_job["RunUserPrecommandOnLoginNode"] = "true"

In [10]:
# - set some environment variables
# - run the python code using interpreter embedded in container
simulation_job["Executable"] = """module use $OTHERSTAGES
module --force purge
module load Stages/2020
module load GCC Singularity-Tools
unset DISPLAY  # matplotlib may look for a nonexistant display on compute node(s)
export SINGULARITYENV_SCRATCH=  # make container unaware of $SCRATCH area as we are already there - write sim output to simulation script folder
export SINGULARITYENV_PYTHONPATH=/opt/nest/lib/python3.9/site-packages
cd hybridLFPy/examples  # go to examples
srun --mpi=pmi2 singularity exec hybridlfpy.sif python3 -u example_microcircuit.py  # execute simulation
"""

In [11]:
# commands run after job is done
simulation_job["User postcommand"] = "" 
simulation_job["RunUserPostcommandOnLoginNode"] = "true"

In [12]:
simulation_job

{'Resources': {'Queue': 'batch',
  'Nodes': '4',
  'CPUsPerNode': '128',
  'Runtime': '3600'},
 'User precommand': 'module use $OTHERSTAGES\nmodule --force purge\nmodule load Stages/2020\nmodule load GCC Singularity-Tools git\n\ngit clone --branch nest3 https://github.com/INM-6/hybridLFPy.git\ncd hybridLFPy/examples\nsib download --recipe-name hybridlfpy\n',
 'RunUserPrecommandOnLoginNode': 'true',
 'Executable': 'module use $OTHERSTAGES\nmodule --force purge\nmodule load Stages/2020\nmodule load GCC Singularity-Tools\nunset DISPLAY  # matplotlib may look for a nonexistant display on compute node(s)\nexport SINGULARITYENV_SCRATCH=  # make container unaware of $SCRATCH area as we are already there - write sim output to simulation script folder\nexport SINGULARITYENV_PYTHONPATH=/opt/nest/lib/python3.8/site-packages\ncd hybridLFPy/examples  # go to examples\nsrun --mpi=pmi2 singularity exec hybridlfpy.sif python3 -u example_microcircuit.py  # execute simulation\n',
 'User postcommand': ''

In [13]:
# create job
job = site_client.new_job(job_description=simulation_job, inputs=simulation_inputs)

In [14]:
job.working_dir

Storage: https://zam2125.zam.kfa-juelich.de:9112/JUSUF/rest/core/storages/10f72afc-4f0a-4089-8fd5-436057a58c38-uspace

In [15]:
# wait while job is running and track progress
while job.is_running():
    clear_output()
    # STDERR output (if any)
    try:
        stderr = job.working_dir.stat('stderr')
        #pprint(stderr.raw().readlines()[:5])
        pprint(stderr.raw().readlines()[-5:])
    except:
        print('no stderr file to read yet')
    # STDOUT output
    try:
        stdout = job.working_dir.stat('stdout')
        #pprint(stdout.raw().readlines()[:10])
        pprint(stdout.raw().readlines()[-10:])
    except:
        print('no stdout file to read yet')
    sleep(10)

[b'--No graphics will be displayed.\n',
 b'Translating alphaisyn.mod into alphaisyn.c\n',
 b'Translating expsyni.mod into expsyni.c\n',
 b'Thread Safe\n',
 b'Thread Safe\n']
[b'Execution time: 2031.157 seconds\n',
 b'Execution time: 2030.633 seconds\n',
 b'Execution time: 2030.941 seconds\n',
 b'Execution time: 2030.682 seconds\n',
 b'Execution time: 2030.646 seconds\n',
 b'Execution time: 2030.647 seconds\n',
 b'Execution time: 2030.911 seconds\n',
 b'Execution time: 2030.710 seconds\n',
 b'Execution time: 2030.797 seconds\n',
 b'Execution time: 2030.652 seconds\n']


In [16]:
job.working_dir.listdir().keys()

dict_keys(['UNICORE_SCRIPT_EXIT_CODE', 'stderr', 'hybridLFPy/', 'UNICORE_Job_1606169657920', 'stdout', '.UNICORE_POST_0/', '.UNICORE_PRE_0/', 'bss_submit_1606169657920'])

In [17]:
# download simulation output
job.working_dir.stat('hybridLFPy/examples/simulation_output_example_microcircuit.tar'
                     ).download('simulation_output_example_microcircuit.tar')

In [18]:
# kill job, clean up files on the remote
job.delete()

## Investigate simulation output
Here we choose to print some pdf file output from the simulation, however all simulation output are available within the same simulation output file hierarchy.

In [19]:
# untar output to folder example_network_output
!tar -xf simulation_output_example_microcircuit.tar

In [20]:
IFrame("./simulation_output_example_microcircuit/figures/layers.pdf", width=800, height=600)

In [21]:
# quick look at e.g., the extracellular potential and spike raster plot
IFrame("./simulation_output_example_microcircuit/figures/cell_models.pdf", width=800, height=600)

In [22]:
IFrame("./simulation_output_example_microcircuit/figures/soma_locations.pdf", width=800, height=600)

In [23]:
IFrame("./simulation_output_example_microcircuit/figures/network_raster.pdf", width=800, height=600)

In [24]:
IFrame("./simulation_output_example_microcircuit/figures/compound_signals.pdf", width=800, height=600)

In [25]:
IFrame("./simulation_output_example_microcircuit/figures/current_dipole_moments.pdf", width=800, height=600)